In [31]:
# coding: utf-8
import requests,re
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords

In [52]:
ListShazam = pd.read_csv('shazam1.csv',sep=',')
ListShazam['Paroles'] = "vide"
ListShazam['MotsFrequents'] = "vide"

In [69]:
# youtube
def importSoup(url):
    request_headers = {
       "Accept-Language": "en-US,en;q=0.5",
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
       "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
       "Referer": "http://thewgfdgebsite.com",
       "Connection": "keep-alive"
    }
    res = requests.get(url, headers = request_headers)
    if res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup



In [70]:
MAX_RETRIES = 20


for i in range(len(ListShazam)):
    soup = soupParoles = all_links = Paroles = parole = urlParoles = ""
    Title, Artist = ListShazam["Title"][i], ListShazam["Artiste"][i]
    Titlep = Title.replace(' ','+')
    Artistp = Artist.replace(' ','+')
    url = r"https://search.azlyrics.com/search.php?q=" + Titlep + '+' + Artistp
    soup = importSoup(url)
    all_links = map(lambda x : x.attrs['href'] , soup.find_all("a"))
    for a in all_links:
        if re.search(r'https://www.azlyrics.com/lyrics/', a):
            urlParoles = a
            break
    if urlParoles != "":
        soupParoles = importSoup(urlParoles)
        Paroles = soupParoles.find_all("div",attrs={'class':'col-xs-12 col-lg-8 text-center'})
        parole = re.sub('\n',' ',Paroles[0].findAll('div')[6].text)
        parole = re.sub('\r',' ',parole)
        wordcount = Counter(parole.split(' '))
        wc = wordcount.most_common()
        ListShazam['Paroles'][i] = parole
        ListShazam['MotsFrequents'][i] = wc[0][0]
    else:
        ListShazam['Paroles'][i] = "vide"


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [60]:
ListShazam.head()
    

,Unnamed: 0,ID,Title,Artiste,Genre,Sortie,Duree,Note_Moyenne_itunes,#_votes_itunes,#_Shazam_fr,Paroles,MotsFrequents
0,0,402107013,Later Bitches,The Prince Karma,Dance,15 juin 2018,4,3.9,11,3016822,vide,vide
1,1,427101360,Shallow,Lady Gaga & Bradley Cooper,Bande originale,5 oct. 2018,1,4.7,205,2166898,vide,vide
2,2,444295408,Gam Gam,Marnik & SMACK,Dance,26 oct. 2018,3,-,-,125014,vide,vide
3,3,448518908,Somewhere Only We Know,Reneé Dominique,-,-,-,-,-,159784,vide,vide
4,4,462933,Come And Get Your Love,Redbone,Rock,1 nov. 1973,3,-,-,2755753,vide,vide


In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hmann\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [37]:
paroles = ListShazam['Paroles']
stop_words = set(stopwords.words('english')) 
def most_common_words(parole):
    wordcount = Counter(parole.split(' '))
    if not wordcount.most_common()[0][0] in stop_words: 
        list.append(wordcount.most_common()[0][0])
    return list

In [67]:
ListShazam = ListShazam.dropna()
liste_words = ListShazam['Paroles'].apply(most_common_words)

vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide
vide


In [68]:
ListShazam

,Unnamed: 0,ID,Title,Artiste,Genre,Sortie,Duree,Note_Moyenne_itunes,#_votes_itunes,#_Shazam_fr,Paroles,MotsFrequents
0,0,402107013,Later Bitches,The Prince Karma,Dance,15 juin 2018,4,3.9,11,3016822,vide,vide
1,1,427101360,Shallow,Lady Gaga & Bradley Cooper,Bande originale,5 oct. 2018,1,4.7,205,2166898,vide,vide
2,2,444295408,Gam Gam,Marnik & SMACK,Dance,26 oct. 2018,3,-,-,125014,vide,vide
3,3,448518908,Somewhere Only We Know,Reneé Dominique,-,-,-,-,-,159784,vide,vide
4,4,462933,Come And Get Your Love,Redbone,Rock,1 nov. 1973,3,-,-,2755753,vide,vide
5,5,434628736,Thunderclouds,LSD Feat. Sia & Diplo & Labrinth,Pop,9 août 2018,3,4.5,23,3320925,vide,vide
6,6,437624347,Tout Oublier,Angele & Roméo Elvis,Variété française,5 oct. 2018,4,3.9,133,233521,vide,vide
7,7,408576846,Shotgun,George Ezra,Auteur-interprète,23 mars 2018,3,4.8,12,3252188,vide,vide
8,8,440557235,Taki Taki,DJ Snake Feat. Selena Gomez & Ozuna & Cardi B,Dance,28 sept. 2018,3,3.9,32,2796841,vide,vide
9,9,416737862,MIA,Bad Bunny Feat. Drake,Latino,11 oct. 2018,3,-,-,1257227,vide,vide
